<center>  
$$ \huge \textbf{FundusI3: InceptionV3 Model} $$  🏥👁️  
</center>  

**Purpose:**  
- 💡 Classifies fundus images as 'Normal' or 'AbNormal' for Alzheimer's detection.  

**Model Overview:**  
- 🔑 **Backbone:** InceptionV3 — A deep CNN designed for **efficient feature extraction** with factorized convolutions.  

**Techniques Used:**  
- 🔄 **Transfer Learning:** Pre-trained on ImageNet for improved generalization.  
- ⚖️ **Binary Cross-Entropy Loss** for classification.  
- 🚀 **AdamW Optimizer** with weight decay for stability.  
- ⚖️ **Class Weighting** to handle dataset imbalance.  
- 🔎 **Feature Extraction:** Uses **final convolutional block + ‘mixed7’ layer** for enriched representation.  
- ⏱️ **Callbacks:** Early stopping, ReduceLROnPlateau & Model Checkpointing for optimized training.  

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# Dataset Paths
train_dir = "/content/drive/MyDrive/Fundus-ModelData/Train-Imgs"
test_dir = "/content/drive/MyDrive/Fundus-ModelData/Test-Imgs"

# Image Parameters
IMG_SIZE = 224
BATCH_SIZE = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2  # 20% Validation Split
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Compute Class Weights
class_labels = train_generator.class_indices
num_samples = train_generator.samples
class_counts = [789, 405]  # AbNormal, Normal counts

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(list(class_labels.values())),
    y=np.concatenate([np.ones(class_counts[0]), np.zeros(class_counts[1])])
)

class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weight_dict)

# Load InceptionV3 Model
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Extract Features from 'mixed7' and Final Layer
x1 = base_model.get_layer("mixed7").output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model.output
x2 = GlobalAveragePooling2D()(x2)

# Concatenate Features
x = tf.keras.layers.concatenate([x1, x2])
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

# Final Model
model = Model(inputs=base_model.input, outputs=output)

# Freeze Backbone (First Training Phase)
for layer in base_model.layers:
    layer.trainable = False

# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
checkpoint = ModelCheckpoint("FundusI3_best_model.h5", save_best_only=True, monitor="val_loss", mode="min")
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)

callbacks = [checkpoint, early_stop, reduce_lr]

# Train Model (Phase 1: Feature Extraction)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# Unfreeze Some Layers for Fine-Tuning
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
    layer.trainable = True

# Compile Again with Lower LR
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train Model (Phase 2: Fine-Tuning)
history_finetune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# Save Model
model.save("FundusI3_final_model.h5")
print("FundusI3 Model Training Completed.")

**Prediction:**  
- 🧑‍💻 Predict whether new fundus images are 'Normal' or 'AbNormal'.  

**Steps:**  
1. 📂 Load the pre-trained **FundusI3** model.  
2. 🔄 Preprocess the images (resize to 224x224 & normalize pixel values).  
3. 📊 Run the preprocessed images through the **InceptionV3** model for predictions.  
4. 🧐 Display the prediction result (Normal 🟢 or AbNormal 🔴) along with the confidence score.  

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from fpdf import FPDF
from google.colab import drive

# 🚀 Mount Google Drive
drive.mount('/content/drive')

# 🔍 Define paths
model_path = "/content/FundusI3_best_model.h5"  # Update with your saved model path
test_folder = "/content/drive/MyDrive/Fundus-ModelData/Test-Imgs"  # Folder containing subfolders with images
output_pdf_path = "/content/drive/MyDrive/Fundus-ModelData/FundusI3-Report"

# 🧠 Load Trained Model
model = tf.keras.models.load_model(model_path)

# 🖼 Preprocess Image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Resize
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# 📂 Process Images & Predict
results = []
class_counts = {}  # For Pie Chart
confidence_scores = {}  # For Average Confidence Chart
threshold = 0.5  # Sigmoid activation threshold

for subfolder in os.listdir(test_folder):
    subfolder_path = os.path.join(test_folder, subfolder)

    if os.path.isdir(subfolder_path):  # Ensure it's a folder
        class_counts[subfolder] = {"Normal": 0, "AbNormal": 0}
        confidence_scores[subfolder] = []

        for image_name in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_name)

            # Predict class
            img_array = preprocess_image(image_path)
            prediction = model.predict(img_array)[0][0]
            predicted_class = "AbNormal" if prediction > threshold else "Normal"
            confidence_score = round(prediction, 4)

            # Track counts & confidence
            class_counts[subfolder][predicted_class] += 1
            confidence_scores[subfolder].append(confidence_score)

            # Store results
            results.append([image_name, subfolder, predicted_class, confidence_score])

# 📝 Convert results to DataFrame
df = pd.DataFrame(results, columns=["Image Name", "Folder", "Predicted Class", "Confidence Score"])

# 🎨 Generate Data Visualizations
fig, axes = plt.subplots(1, len(class_counts), figsize=(10, 5))
for i, (folder, counts) in enumerate(class_counts.items()):
    labels = counts.keys()
    sizes = counts.values()
    axes[i].pie(sizes, labels=labels, autopct="%1.1f%%", startangle=90, colors=["lightblue", "salmon"])
    axes[i].set_title(f"Class Distribution in {folder}")
plt.savefig("/content/class_distribution.png")

# 📊 Average Confidence per Folder
avg_confidence = {folder: np.mean(scores) for folder, scores in confidence_scores.items()}
plt.figure(figsize=(6, 4))
plt.bar(avg_confidence.keys(), avg_confidence.values(), color="skyblue")
plt.xlabel("Folders")
plt.ylabel("Average Confidence Score")
plt.title("Average Confidence Score per Subfolder")
plt.savefig("/content/avg_confidence.png")

!pip install fpdf

# 📄 Generate PDF Report
class PDF(FPDF):
    def header(self):
        self.set_font("Times", "B", 16)
        self.cell(275, 10, "Fundus Image Classification Report", ln=True, align="C")

    def footer(self):
        self.set_y(-15)
        self.set_font("Times", "I", 10)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

# 📑 Create PDF
pdf = PDF(orientation="L")  # Landscape Mode
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Times", size=12)

# 🏷 Report Header
pdf.set_font("Times", "B", 14)
pdf.cell(0, 10, "Model Details", ln=True)
pdf.set_font("Times", size=12)
pdf.cell(0, 10, f"Model Name: FundusR50", ln=True)
#pdf.cell(0, 10, f"Model Path: {model_path}", ln=True)
pdf.cell(0, 10, "Author: Danish A. G.", ln=True)
pdf.ln(10)

# 📋 Results Table
pdf.set_font("Times", "B", 12)

# Adjusted Column Widths
pdf.cell(140, 10, "Image Name", 1)  # Increased width
pdf.cell(40, 10, "Folder", 1)
pdf.cell(40, 10, "Predicted Class", 1)
pdf.cell(50, 10, "Confidence Score", 1)
pdf.ln()

pdf.set_font("Times", size=12)
for idx, row in df.iterrows():
    pdf.cell(140, 10, row["Image Name"], 1)  # Adjusted width
    pdf.cell(40, 10, row["Folder"], 1)
    pdf.cell(40, 10, row["Predicted Class"], 1)
    pdf.cell(50, 10, str(row["Confidence Score"]), 1)
    pdf.ln()

# 📊 Insert Pie Charts
pdf.ln(10)
pdf.cell(0, 10, "Class Distribution", ln=True, align="L")
pdf.image("/content/class_distribution.png", x=10, w=250)

# 📉 Insert Average Confidence Score Chart
pdf.ln(10)
pdf.cell(0, 10, "Average Confidence Score per Subfolder", ln=True, align="L")
pdf.image("/content/avg_confidence.png", x=10, w=250)

# 💾 Save PDF to Drive
pdf.output(output_pdf_path)
print(f"✅ Report saved at: {output_pdf_path}")